In [1]:
from __future__ import annotations

import cProfile
import pstats
from asyncio import sleep

In [2]:
from py_research.db import (
    DataSource,
    TableMap,
    SubTableMap,
    SubMap,
    SelIndex,
    DataBase
)

In [3]:
from test_db_schema import (
    Search,
    Project,
    Task,
    User,
    Membership,
    Organization,
    SearchResult,
    TestSchema,
)

In [4]:
from typing import Any


async def dummy_loader(obj: dict[str, Any]) -> dict:
    await sleep(0.2)
    return {
        'name': obj["name"],
        'role': obj["role"],
        'age': obj["age"] + 1
    }

In [5]:
data_source = DataSource(
    target=Search,
    push={
        "resultCount": Search.result_count,
        "search": Search.term,
        "results": SubTableMap(
            target=Search.results,
            rel_map=TableMap(push={SearchResult.score}),
            push={
                "project_name": Project.name,
                "project_start": Project.start,
                "project_end": Project.end,
                "project_status": Project.status,
                "tasks": SubTableMap(
                    target=Project.tasks,
                    push={
                        "task_name": Task.name,
                        "task_assignees": SubTableMap(
                            target=Task.assignees,
                            push=User.name,
                            match_by=[User.name],
                        ),
                        "task_status": Task.status,
                    },
                ),
                "members": SubTableMap(
                    target=Project.members,
                    loader=dummy_loader,
                    push={User.name, User.age},
                    match_by=[User.name],
                    rel_map=TableMap(
                        push={
                            Membership.role,
                        },
                    ),
                ),
            },
            pull={
                Project.org: SubMap(
                    push={
                        "organization_name": Organization.name,
                        "organization_address": Organization.address,
                        "organization_city": Organization.city,
                        "organization_countries": Organization.countries,
                    },
                    match_by=[Organization.name, Organization.city],
                ),
                Project.number: SelIndex()
            },
        ),
    },
)

In [6]:
from datetime import date


data = {
    "resultCount": 3,
    "search": "test",
    "results": [
        {
            "score": 0.9,
            "project_name": "baking cake",
            "project_start": date(2020, 1, 1),
            "project_end": date(2020, 1, 4),
            "project_status": "done",
            "organization_name": "Bakery",
            "organization_address": "Main Street 1",
            "organization_city": "Bakerville",
            "organization_countries": ["USA", "Mexico", "Canada"],
            "tasks": [
                {
                    "task_name": "task1",
                    "task_assignees": ["John"],
                    "task_status": "todo",
                },
                {
                    "task_name": "task2",
                    "task_assignees": ["John"],
                    "task_status": "todo",
                },
                {
                    "task_name": "task3",
                    "task_assignees": ["Jane"],
                    "task_status": "done",
                },
            ],
            "members": [
                {"name": "John", "role": "baker", "age": 30},
                {"name": "John", "role": "manager", "age": 40},
            ],
        },
        {
            "score": 0.8,
            "project_name": "cleaning shoes",
            "project_start": date(2020, 1, 2),
            "project_end": date(2020, 1, 5),
            "project_status": "done",
            "organization_name": "Shoe Shop",
            "organization_address": "Main Street 2",
            "organization_city": "Shoetown",
            "organization_countries": ["USA", "Mexico"],
            "tasks": [
                {
                    "task_name": "task4",
                    "task_assignees": ["John"],
                    "task_status": "todo",
                },
                {
                    "task_name": "task5",
                    "task_assignees": ["Jane"],
                    "task_status": "todo",
                },
            ],
            "members": [
                {"name": "John", "role": "cleaner", "age": 25},
                {"name": "Jane", "role": "manager", "age": 35},
            ],
        },
        {
            "score": 0.7,
            "project_name": "fixing cars",
            "project_start": date(2020, 1, 3),
            "project_end": date(2020, 1, 6),
            "project_status": "started",
            "organization_name": "Car Shop",
            "organization_address": "Main Street 3",
            "organization_city": "Cartown",
            "organization_countries": ["USA", "Canada"],
            "tasks": [
                {
                    "task_name": "task6",
                    "task_assignees": ["John"],
                    "task_status": "todo",
                }
            ],
            "members": [
                {"name": "John", "role": "mechanic", "age": 45},
                {"name": "Jane", "role": "manager", "age": 55},
                {"name": "Jack", "role": "manager", "age": 65},
            ],
        },
    ],
}

In [7]:
import warnings

with warnings.catch_warnings():
  warnings.simplefilter("error")
  with cProfile.Profile() as pr:
    db = DataBase(schema=TestSchema)
    rec = await data_source.load([data], db=db)

    stats = pstats.Stats(pr)

In [8]:
db[User].values()


[
    User({'name': 'John', 'age': 31, '_id': '6b629eb3-4827-4e8c-8766-f8b666ba6453'}),
    User({'name': 'Jane', 'age': 56, '_id': 'b9e6978d-639e-4cff-95a1-41f89e70651b'}),
    User({'name': 'Jack', 'age': 66, '_id': 'bfacf9e2-eed9-462f-b8a0-2c1d4cb12752'})
]

In [9]:
db[Search.term].values()

['test']

In [10]:
list(db[Search.result_count].items())

[('test', 3)]

In [11]:
list(db[Search.results.x.org].items())


[
    (
        ('test', 0),
        Organization({'name': 'Car Shop', 'address': 'Main Street 3', 'city': 'Cartown', '_id': 'dec899ce-1ce4-426e-981c-48f6f49f0964'})
    ),
    (
        ('test', 2),
        Organization({'name': 'Shoe Shop', 'address': 'Main Street 2', 'city': 'Shoetown', '_id': 'd6101149-b122-46e0-8719-fc6912abd187'})
    ),
    (
        ('test', 1),
        Organization({'name': 'Bakery', 'address': 'Main Street 1', 'city': 'Bakerville', '_id': '48526839-545c-4fe2-baa2-8ee67371cfe0'})
    )
]

In [12]:
list(db[Search.results].items())[0][1].start.year

2020

In [13]:
db[Search.results].df()

test_db_schema_Search_term,end,number,start,status,org__id,name
str,date,i64,date,str,str,str
"""test""",2020-01-04,0,2020-01-01,"""done""","""dec899ce-1ce4-426e-981c-48f6f4…","""baking cake"""
"""test""",2020-01-06,2,2020-01-03,"""started""","""d6101149-b122-46e0-8719-fc6912…","""fixing cars"""
"""test""",2020-01-05,1,2020-01-02,"""done""","""48526839-545c-4fe2-baa2-8ee673…","""cleaning shoes"""


In [14]:
db[Organization].df()

test_db_schema_Organization__id,address,name,city,_id
str,str,str,str,str
"""48526839-545c-4fe2-baa2-8ee673…","""Main Street 1""","""Bakery""","""Bakerville""","""48526839-545c-4fe2-baa2-8ee673…"
"""d6101149-b122-46e0-8719-fc6912…","""Main Street 2""","""Shoe Shop""","""Shoetown""","""d6101149-b122-46e0-8719-fc6912…"
"""dec899ce-1ce4-426e-981c-48f6f4…","""Main Street 3""","""Car Shop""","""Cartown""","""dec899ce-1ce4-426e-981c-48f6f4…"


In [15]:
db.describe()


{
    'schema': {
        'package': 'py-research',
        'module': 'test_db_schema',
        'object': 'TestSchema',
        'package_version': '^5.1.1.dev133+gf6dad64.d20250116',
        'repo': 'https://github.com/cloudlane-one/py-research.git',
        'repo_revision': '1280f14b4c4185cbf2acb888e84301914568fcb1',
        'module_path': WindowsPath('exp/test_db_schema.py'),
        'module_dirty': False,
        'docs_url': 'https://cloudlane-one.github.io/py-research/'
    },
    'contents': {
        'records': {
            'test_db_schema.Task': 6,
            'test_db_schema.Organization': 3,
            'test_db_schema.Search': 1,
            'test_db_schema.User': 3,
            'test_db_schema.Project': 3
        },
        'arrays': {'test_db_schema_Organization_countries': 8},
        'relations': {'test_db_schema.SearchResult': 3, 'test_db_schema.Membership': 7}
    },
    'backend': None
}

In [16]:
db[SearchResult._table[SearchResult.score.isin(slice(0.81, 0.9))]].df()

test_db_schema_SearchResult__id,score,_from_term,_id,_to_number
str,f64,str,str,i64
"""4376575091""",0.9,"""test""","""4376575091""",0


In [17]:
db[Organization.countries].df()

test_db_schema_Organization__id,test_db_schema_Organization_725689__id,_value
str,i64,str
"""dec899ce-1ce4-426e-981c-48f6f4…",0,"""USA"""
"""dec899ce-1ce4-426e-981c-48f6f4…",1,"""Mexico"""
"""dec899ce-1ce4-426e-981c-48f6f4…",2,"""Canada"""
"""48526839-545c-4fe2-baa2-8ee673…",0,"""USA"""
"""48526839-545c-4fe2-baa2-8ee673…",1,"""Mexico"""
"""d6101149-b122-46e0-8719-fc6912…",0,"""USA"""
"""d6101149-b122-46e0-8719-fc6912…",1,"""Canada"""
"""48526839-545c-4fe2-baa2-8ee673…",2,"""Canada"""


In [18]:
stats.sort_stats("cumulative")
stats.print_stats("py_research")

         482552 function calls (448717 primitive calls) in 0.830 seconds

   Ordered by: cumulative time
   List reduced from 2280 to 157 due to restriction <'py_research'>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
       52    0.000    0.000    0.304    0.006 C:\Users\lworm\Repos\py-research\src\py_research\db\datasources.py:697(_load_rec_from_item)
       52    0.002    0.000    0.302    0.006 C:\Users\lworm\Repos\py-research\src\py_research\db\datasources.py:572(_load_record)
       56    0.000    0.000    0.284    0.005 C:\Users\lworm\Repos\py-research\src\py_research\db\datasources.py:838(load)
       19    0.000    0.000    0.275    0.014 C:\Users\lworm\Repos\py-research\src\py_research\db\databases.py:1467(__ior__)
       19    0.001    0.000    0.275    0.014 C:\Users\lworm\Repos\py-research\src\py_research\db\databases.py:2903(_mutate)
   168/57    0.001    0.000    0.272    0.005 C:\Users\lworm\Repos\py-research\src\py_research\db\datasources.py